In [220]:
import polars as pl
import altair as alt

In [221]:
df = pl.read_parquet('data/papezstvo.parquet', use_pyarrow=True)

In [222]:
import geopy.distance

def vzdalenost_od_rima(souradnice):
    try:
        return geopy.distance.geodesic((souradnice['latitude'],souradnice['longitude']), {41.893056,12.482778}).kilometers
    except:
        None

df = df.with_columns(pl.col('narozeni_souradnice').map_elements(vzdalenost_od_rima).alias('vzdalenost_od_rima'))

ColumnNotFoundError: narozeni_souradnice

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'with_columns' <---
DF ["vatikan_poradi", "vatikan_jmeno", "vatikan_pontifikat_zacatek", "vatikan_pontifikat_konec"]; PROJECT */8 COLUMNS; SELECTION: None

In [ ]:
def stoleti(datum):
    return f"{int(str(datum)[0:2]) + 1}. století"

In [ ]:
def cislo_stoleti(datum):
    return int(str(datum)[0:2]) + 1

In [ ]:
df.select(pl.col("pontifikat_zacatek")).to_series().to_list()[0]

In [ ]:
stoleti(df.select(pl.col("pontifikat_zacatek")).to_series().to_list()[0])

In [ ]:
df.with_columns(pl.col("pontifikat_zacatek").map_elements(stoleti).alias("stoleti"))

In [ ]:
def rok(datum):
    return int(str(datum)[2:4])

In [ ]:
do_grafu = df.with_columns(
    pl.col("pontifikat_zacatek").map_elements(cislo_stoleti).alias("stoleti")
).with_columns(
    pl.col("pontifikat_zacatek").map_elements(rok).cast(pl.Int8).alias('rok_zacatku')
).with_columns(
    pl.col("pontifikat_konec").map_elements(rok).cast(pl.Int8).alias('rok_konce')
)

maska = do_grafu["rok_konce"] < do_grafu["rok_zacatku"]
opakovane = do_grafu.filter(maska).with_columns(
    pl.lit(0).cast(pl.Int8).alias("rok_zacatku")).with_columns(pl.col('stoleti') + 1
)

do_grafu_1 = pl.concat([do_grafu, opakovane], how="vertical").sort(by='pontifikat_zacatek')

poradi_stoleti = ['1. století',
          '2. století',
          '3. století',
          '4. století', 
          '5. století',
        '6. století',
          '7. století',
           '8. století',
          '9. století',
          '10. století',
 '11. století',
 '12. století',
          '13. století',
 '14. století',
 
 '15. století',
 '16. století',
 '17. století',
 '18. století',
 '19. století',
          '20. století',
 '21. století', 
]

poradi_papezu = do_grafu.sort(by='pontifikat_zacatek').select(pl.col('jmeno')).to_series().to_list()

In [ ]:
alt.Chart(
    do_grafu_1.with_columns(pl.col("wikihesla").map_elements(len)).to_pandas()
).mark_line(
).encode(
    alt.X("rok_zacatku:Q"),
    alt.X2("rok_konce:Q"),
    alt.Color("jmeno:N", scale=alt.Scale(range=['#AF4752', '#01A1BC', '#F1AFC5', '#F89E43', '#A1D3D9', '#E6B095', '#9CB872', '#B3CBD2']), sort=poradi_papezu, legend=None),
    alt.Row('stoleti:N', header=alt.Header(labelOrient='left',labelAngle=0), spacing=5),
    alt.Size('wikihesla:Q', scale=alt.Scale(range=[0.01,10]), legend=None)
).configure_view(stroke='transparent')

In [ ]:
alt.Chart(
    do_grafu.with_columns(pl.col("wikihesla").map_elements(len)).unique(subset=["jmeno"]).to_pandas()
).mark_line(
).encode(
    alt.X("pontifikat_zacatek:T"),
    alt.X2("pontifikat_konec:T"),
    alt.Color("jmeno:N", scale=alt.Scale(range=['#AF4752', '#01A1BC', '#F1AFC5', '#F89E43', '#A1D3D9', '#E6B095', '#9CB872', '#B3CBD2']), sort=poradi_papezu, legend=None),
    alt.Size('wikihesla:Q', scale=alt.Scale(range=[0.01,10]), legend=None)
).configure_view(stroke='transparent')

In [ ]:
do_grafu.filter(pl.col('stoleti') == 17).head(10)

In [ ]:
rok(df.select(pl.col("pontifikat_zacatek")).to_series().to_list()[3])

In [ ]:
do_grafu_2 = df.with_columns(
    (pl.col("pontifikat_zacatek") - pl.col("narozeni_datum")).alias("nastupni_vek")
).with_columns(
    pl.col("nastupni_vek").dt.total_days() / 365.25
).with_columns(
    ((pl.col("umrti_datum") - pl.col("pontifikat_zacatek")).dt.total_days() / 365.25).alias("zivot_po_zvoleni")
).with_columns(((pl.col('pontifikat_konec') - pl.col('narozeni_datum')).dt.total_days() / 365).alias('konecny_vek'))

In [ ]:
do_grafu_2

In [ ]:
df.tail(100).select(pl.col('vzdalenost_od_rima').quantile(0.9))

In [ ]:
alt.Chart(
    do_grafu_2.tail(100).filter(pl.col("narozeni_presnost") >= 9).to_pandas(),
    width=300
).mark_bar(width=2
).encode(
    alt.X("pontifikat_zacatek:T"),
    alt.Y("nastupni_vek:Q"),
    alt.Y2("konecny_vek:Q"),
    alt.Color("vzdalenost_kategorie:N")
).configure_view(stroke='transparent')

In [ ]:
do_grafu_2

In [ ]:
alt.Chart(
    do_grafu_2.to_pandas(), width=300
).mark_bar(
).encode(
    alt.X("pontifikat_zacatek:T"),
    alt.X2("pontifikat_konec:T"),
    alt.Color("narozeni_presnost:N"),
).configure_view(stroke='transparent')

In [ ]:
do_grafu_2.with_columns(pl.col('wikihesla').map_elements(len))

In [ ]:
alt.Chart(
    do_grafu_2.with_columns(pl.col('wikihesla').map_elements(len)).to_pandas(), width=300
).mark_bar(
).encode(
    alt.X("pontifikat_zacatek:T"),
    alt.X2("pontifikat_konec:T"),
    alt.Y("wikihesla:Q"),
).configure_view(stroke='transparent')

In [ ]:
do_grafu_2.filter(pl.col("pontifikat_zacatek").dt.year() > 1600).group_by_dynamic(index_column="pontifikat_zacatek", every="50y").agg(pl.col("jmeno").unique().len()).sort(by="jmeno")

In [ ]:
df.tail(100).sort(by='vzdalenost_od_rima')

In [ ]:
alt.Chart(df.filter(~pl.col('vzdalenost_od_rima').is_null()).to_pandas()).mark_bar().encode(alt.X('pontifikat_zacatek:T'),alt.Y('vzdalenost_od_rima:Q'))

In [ ]:
df.filter(pl.col("narozeni_presnost") != 11).tail(5)

In [ ]:
df.filter(pl.col("krest_datum").is_not_null()).tail(5)

In [ ]:
df.filter(pl.col("umrti_kategorie") != "přirozená smrt").tail(5)

In [ ]:
df.filter(pl.col("umrti_kategorie").is_null()).tail(5)

## Poslední…

In [ ]:
df.filter(pl.col("vzdalenost_od_rima") < 25).sort(by="pontifikat_zacatek",descending=True).head(1)

In [ ]:
df.filter(pl.col("pontifikat_zacatek").dt.year() < 2010).drop_nulls(subset=['vzdalenost_od_rima']).sort(by="vzdalenost_od_rima",descending=True).head(3)

In [ ]:
do_grafu_2.sort(by='pontifikat_zacatek',descending=True).filter(pl.col('nastupni_vek') < 60).head(1)

In [ ]:
do_grafu_2.sort(by='pontifikat_zacatek',descending=True).filter(pl.col('nastupni_vek') < 55).head(1)

In [ ]:
do_grafu_2.sort(by='pontifikat_zacatek',descending=True).filter(pl.col('nastupni_vek') < 50).head(1)

In [ ]:
do_grafu_2.sort(by='pontifikat_zacatek',descending=True).filter(pl.col('nastupni_vek') < 45).head(1)

In [ ]:
do_grafu_2.sort(by='pontifikat_zacatek',descending=True).filter(pl.col('nastupni_vek') < 35).head(1)

In [ ]:
len(df)

## Kolik za život

In [ ]:
len(df.filter(pl.col("pontifikat_zacatek").dt.year() >= 2025-45))

In [ ]:
rocniky = []
for rok in range(50, 2025):
    rocnik = {}
    rocnik['narozeni'] = rok
    for let in [20,30,40,45,50,60,70,80,90,100]:
        rocnik[f'papezu_do_{let}'] = len(df.filter((pl.col("pontifikat_zacatek").dt.year() >= rok-let) & (pl.col("pontifikat_zacatek").dt.year() <= rok)))
    rocniky.append(rocnik)

In [ ]:
pl.DataFrame(rocniky)

In [ ]:
pl.DataFrame(rocniky).filter(pl.col('narozeni') >= 100).sort(by='papezu_do_45')

In [ ]:
pl.DataFrame(rocniky).filter(pl.col('narozeni') >= 100).filter(pl.col('papezu_do_45') <= 2).sort(by='narozeni')

In [ ]:
pl.DataFrame(rocniky).filter(pl.col('narozeni') >= 100).filter(pl.col('papezu_do_40') <= 2).sort(by='narozeni')

In [ ]:
alt.Chart(
    pl.DataFrame(rocniky).sort(by='narozeni'),
    width=350,
    title="Kolik papežů zažili 40letí a 80letí"
).mark_line().encode(
    alt.X("narozeni:Q"),
    alt.Y("papezu_do_40:Q"),
    alt.Y2("papezu_do_80:Q")
)

In [ ]:
alt.Chart(
    pl.DataFrame(rocniky).sort(by='narozeni'),
    width=350,
    title="Kolik papežů zažili 40letí a 80letí"
).mark_line().encode(
    alt.X("narozeni:Q"),
    alt.Y("papezu_do_50:Q")
)

In [ ]:
pl.DataFrame(rocniky).filter(pl.col('narozeni') >= 100).filter(pl.col('papezu_do_40') >= 6).sort(by='narozeni')

In [ ]:
pl.DataFrame(rocniky).filter(pl.col('narozeni') >= 100).filter(pl.col('papezu_do_40') <= 1).tail(20)